In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!wget "https://raw.githubusercontent.com/akshaycholapurath/DeepNNwithTensorflow/master/helper_function.py"

--2021-05-21 15:46:38--  https://raw.githubusercontent.com/akshaycholapurath/DeepNNwithTensorflow/master/helper_function.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10475 (10K) [text/plain]
Saving to: ‘helper_function.py’

helper_function.py  100%[===================>]  10.23K  --.-KB/s    in 0s      

2021-05-21 15:46:38 (104 MB/s) - ‘helper_function.py’ saved [10475/10475]



Import the helper functions

In [3]:
from helper_function import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

In [4]:
#Get the text dataset - Tweets with labels disaster or not disaster
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"


--2021-05-21 15:46:39--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.63.128, 142.250.31.128, 172.217.164.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.63.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.008s  

2021-05-21 15:46:39 (69.5 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data('/content/nlp_getting_started.zip')

In [6]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
train_df = train_df.sample(frac=1,random_state=42)

In [9]:
train_df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
train_df.describe(include = 'all')

,id,keyword,location,text,target
count,7613.000000,7552,5080,7613,7613.00000
unique,NaN,221,3341,7503,NaN
top,NaN,fatalities,USA,11-Year-Old Boy Charged With Manslaughter of T...,NaN
freq,NaN,45,104,10,NaN
mean,5441.934848,NaN,NaN,NaN,0.42966
std,3137.116090,NaN,NaN,NaN,0.49506
min,1.000000,NaN,NaN,NaN,0.00000
25%,2734.000000,NaN,NaN,NaN,0.00000
50%,5408.000000,NaN,NaN,NaN,0.00000
75%,8146.000000,NaN,NaN,NaN,1.00000


In [11]:
import random

random_index = random.randint(0,len(train_df)-5)
for row in train_df[['text','target']][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"Target:{target}","(real dissaster)" if target >0 else "(not real disaster)")
  print(f"Text:\n {text} \n")
  print("---\n")

Target:1 (real dissaster)
Text:
 Holy moly that was crushed. 

---

Target:1 (real dissaster)
Text:
 @gilmanrocks7 wow. Where is this? For the rest of the summer I usually have to worry about forest fires. Calgary is about 8 hours away. 

---

Target:0 (not real disaster)
Text:
 US wants future first responders to be more high-tech - http://t.co/1VI2RNbK2i 

---

Target:0 (not real disaster)
Text:
 Quality Metrics Penalties May Harm Patient Care PCPs Say: Primary care physicians generally hold positive vi... http://t.co/TdwprVB04y 

---

Target:0 (not real disaster)
Text:
 RT RoadID: Thanks to Alex for his story &amp; to all first responders for being there when we need you. Û_ http://t.co/HikDC1fM2F 

---



In [12]:
X = train_df['text'].to_numpy()
y = train_df['target'].to_numpy()

In [13]:
y

array([1, 0, 1, ..., 0, 1, 1])

In [14]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=42)

In [15]:
x_train[:10],y_train[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

In [16]:
len(x_train),len(y_train),len(x_test),len(y_test)

(6851, 6851, 762, 762)

# Tokenize Data

In [17]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [18]:
round(sum([len(i.split()) for i in x_train])/len(x_train))

15

In [19]:
max_vocab_size=10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_size,
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',ngrams=None,
                                    output_mode = 'int',output_sequence_length=max_length,
                                    pad_to_max_tokens=True)

In [20]:
#Fit text_vectorizer to training data
text_vectorizer.adapt(x_train)

In [21]:
x_train

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

In [22]:
sample = 'This is a really good exercise'
text_vectorizer([sample])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 19,   9,   3, 169, 136,   1,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [23]:
#Get the unique words and their tokens
words = text_vectorizer.get_vocabulary()
words[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [24]:
words[-5:]

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

# Embedding

In [25]:
from tensorflow.keras import layers

#converts each int token into a 128 dim vector randomly initialized

embedding = layers.Embedding(input_dim = max_vocab_size,
                             output_dim = 128,
                             input_length=max_length)

# Model_0 :Naive Bayes with TF-IDF encoder

Base model - Naive Bayes with TF-IDF encoder

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
          ('tfidf',TfidfVectorizer()),
          ('clf',MultinomialNB())
])

model_0.fit(x_train,y_train)



Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [27]:
model0_score = model_0.score(x_test,y_test)
model0_score

0.7926509186351706

In [28]:
model0_preds = model_0.predict(x_test)
model0_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [29]:
#Creating a classification metrics function
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  acc = accuracy_score(y_true,y_pred)*100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = { "Accuracy":acc,
                    "Precision":model_precision,
                    "Recall" :model_recall,
                     "F1_score":model_f1 
  }
  return model_results


In [30]:
calculate_results(y_test,model0_preds)

{'Accuracy': 79.26509186351706,
 'F1_score': 0.7862189758049549,
 'Precision': 0.8111390004213173,
 'Recall': 0.7926509186351706}

# Model_1:Dense NN model

Dense NN model

In [31]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Dense(10,activation='relu')(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation='sigmoid')(x)

model_1 = tf.keras.Model(inputs,outputs)

In [32]:
model_1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
dense (Dense)                (None, 15, 10)            1290      
_________________________________________________________________
global_average_pooling1d (Gl (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 1,281,301
Trainable params: 1,281,301
Non-trainable params: 0
___________________________________________________

In [33]:
model_1.compile(optimizer=tf.keras.optimizers.Adam(),
                loss='binary_crossentropy',
                metrics = ['accuracy'])

In [34]:
history_1  = model_1.fit(x_train,y_train,epochs=10,
                         validation_data = (x_test,y_test),
                         callbacks=[create_tensorboard_callback(dir_name='training_logs',
                                                               experiment_name='model_1_dense')]
                        )

Saving TensorBoard log files to: training_logs/model_1_dense/20210521-154646
Epoch 1/10
215/215 [==============================] - 7s 19ms/step - loss: 0.6508 - accuracy: 0.6310 - val_loss: 0.4920 - val_accuracy: 0.7769
Epoch 2/10
215/215 [==============================] - 3s 14ms/step - loss: 0.4025 - accuracy: 0.8363 - val_loss: 0.4600 - val_accuracy: 0.7874
Epoch 3/10
215/215 [==============================] - 3s 15ms/step - loss: 0.2866 - accuracy: 0.8857 - val_loss: 0.4969 - val_accuracy: 0.7953
Epoch 4/10
215/215 [==============================] - 3s 15ms/step - loss: 0.2288 - accuracy: 0.9074 - val_loss: 0.4946 - val_accuracy: 0.7835
Epoch 5/10
215/215 [==============================] - 3s 14ms/step - loss: 0.1769 - accuracy: 0.9340 - val_loss: 0.5338 - val_accuracy: 0.7900
Epoch 6/10
215/215 [==============================] - 3s 14ms/step - loss: 0.1453 - accuracy: 0.9450 - val_loss: 0.5859 - val_accuracy: 0.7822
Epoch 7/10
215/215 [==============================] - 3s 15ms/ste

In [35]:
model_1_pred = tf.squeeze(tf.round(model_1.predict(x_test)))
model_1_pred

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0

In [36]:
calculate_results(y_test,model_1_pred)

{'Accuracy': 76.9028871391076,
 'F1_score': 0.7666669868908192,
 'Precision': 0.7711368167182066,
 'Recall': 0.7690288713910761}

In [37]:
weights = model_1.get_layer('embedding').get_weights()[0]
vocab = text_vectorizer.get_vocabulary()

# Model_2 :LSTM model

LSTM model

In [38]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(16,return_sequences=True,dropout=.5)(x)
x = layers.LSTM(16,dropout=.5)(x)
x = layers.Dense(10,activation='relu')(x)
outputs = layers.Dense(1,activation='sigmoid')(x)

model_2 = tf.keras.Model(inputs,outputs)

In [39]:
model_2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 15, 16)            9280      
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                170       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11  

In [40]:
model_2.compile(optimizer=tf.keras.optimizers.Adam(),
                loss='binary_crossentropy',
                metrics = ['accuracy'])

In [41]:
history_2  = model_2.fit(x_train,y_train,epochs=5,
                         validation_data = (x_test,y_test),
                         callbacks=[create_tensorboard_callback(dir_name='training_logs',
                                                               experiment_name='model_2_rnn')]
                        )

Saving TensorBoard log files to: training_logs/model_2_rnn/20210521-154722
Epoch 1/5
215/215 [==============================] - 37s 25ms/step - loss: 0.3994 - accuracy: 0.8428 - val_loss: 0.6814 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0896 - accuracy: 0.9701 - val_loss: 0.7670 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0852 - accuracy: 0.9706 - val_loss: 0.7803 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0725 - accuracy: 0.9719 - val_loss: 0.9375 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0627 - accuracy: 0.9764 - val_loss: 1.0292 - val_accuracy: 0.7638


In [42]:
model_2_pred = tf.squeeze(tf.round(model_2.predict(x_test)))
model_2_pred

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0

In [43]:
calculate_results(y_test,model_2_pred)

{'Accuracy': 76.37795275590551,
 'F1_score': 0.761768588939328,
 'Precision': 0.7649838845988884,
 'Recall': 0.7637795275590551}

# Model_3 : GRU model

GRU model

In [44]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(16)(x)
x = layers.Dense(10,activation='relu')(x)
outputs = layers.Dense(1,activation='sigmoid')(x)

model_3 = tf.keras.Model(inputs,outputs)

In [45]:
model_3.compile(optimizer=tf.keras.optimizers.Adam(),
                loss='binary_crossentropy',
                metrics = ['accuracy'])

In [46]:
history_3  = model_3.fit(x_train,y_train,epochs=5,
                         validation_data = (x_test,y_test),
                         callbacks=[create_tensorboard_callback(dir_name='training_logs',
                                                               experiment_name='model_3_rnn')]
                        )

Saving TensorBoard log files to: training_logs/model_3_rnn/20210521-155158
Epoch 1/5
215/215 [==============================] - 6s 20ms/step - loss: 0.4695 - accuracy: 0.7602 - val_loss: 0.7325 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0749 - accuracy: 0.9720 - val_loss: 0.7838 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0627 - accuracy: 0.9767 - val_loss: 0.9111 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0482 - accuracy: 0.9783 - val_loss: 0.9245 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0428 - accuracy: 0.9825 - val_loss: 0.9994 - val_accuracy: 0.7638


In [47]:
model_3.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru (GRU)                    (None, 16)                7008      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                170       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 1,287,189
Trainable params: 1,287,189
Non-trainable params: 0
_________________________________________________

In [48]:
model_3_pred = tf.squeeze(tf.round(model_3.predict(x_test)))
model_3_pred

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
       1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [49]:
calculate_results(y_test,model_3_pred)

{'Accuracy': 76.37795275590551,
 'F1_score': 0.762372083327101,
 'Precision': 0.7640630933275061,
 'Recall': 0.7637795275590551}

# Model_4 : Bi-directional LSTM

Bi-directional LSTM

In [67]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(32,dropout=.5,return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(32,dropout=.5))(x)
outputs = layers.Dense(1,activation='sigmoid')(x)

model_4 = tf.keras.Model(inputs,outputs)

In [68]:
model_4.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.01),
                loss='binary_crossentropy',
                metrics = ['accuracy'])

In [69]:
history_4  = model_4.fit(x_train,y_train,epochs=5,
                         validation_data = (x_test,y_test),
                         callbacks=[create_tensorboard_callback(dir_name='training_logs',
                                                               experiment_name='model_4_rnn')]
                        )

Saving TensorBoard log files to: training_logs/model_4_rnn/20210521-162353
Epoch 1/5
215/215 [==============================] - 12s 31ms/step - loss: 0.2159 - accuracy: 0.9096 - val_loss: 0.7042 - val_accuracy: 0.7638
Epoch 2/5
215/215 [==============================] - 5s 22ms/step - loss: 0.1245 - accuracy: 0.9570 - val_loss: 0.6960 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 5s 21ms/step - loss: 0.0921 - accuracy: 0.9667 - val_loss: 0.9279 - val_accuracy: 0.7493
Epoch 4/5
215/215 [==============================] - 4s 21ms/step - loss: 0.0700 - accuracy: 0.9733 - val_loss: 0.9478 - val_accuracy: 0.7467
Epoch 5/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0638 - accuracy: 0.9702 - val_loss: 1.0967 - val_accuracy: 0.7520


In [66]:
model_4_pred = tf.squeeze(tf.round(model_4.predict(x_test)))
model_4_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [65]:
calculate_results(y_test,model_4_pred)

{'Accuracy': 76.24671916010499,
 'F1_score': 0.7599672431611975,
 'Precision': 0.7645157014219752,
 'Recall': 0.7624671916010499}

# Model_5 : 1D Conv NN

In [82]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(64,3,activation='relu',padding='same')(x)
x = layers.Conv1D(64,3,activation='relu',padding='same')(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation='sigmoid')(x)

model_5 = tf.keras.Model(inputs,outputs)

In [83]:
model_5.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 15, 64)            24640     
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 15, 64)            12352     
_________________________________________________________________
global_average_pooling1d_2 ( (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 65 

In [84]:
model_5.compile(optimizer=tf.keras.optimizers.Adam(),
                loss='binary_crossentropy',
                metrics = ['accuracy'])

In [85]:
history_5  = model_5.fit(x_train,y_train,epochs=5,
                         validation_data = (x_test,y_test),
                         callbacks=[create_tensorboard_callback(dir_name='training_logs',
                                                               experiment_name='model_5conv1d')]
                        )

Saving TensorBoard log files to: training_logs/model_5conv1d/20210521-163459
Epoch 1/5
215/215 [==============================] - 5s 18ms/step - loss: 0.1932 - accuracy: 0.9456 - val_loss: 1.2733 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0387 - accuracy: 0.9841 - val_loss: 1.4326 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0389 - accuracy: 0.9829 - val_loss: 1.6888 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0357 - accuracy: 0.9832 - val_loss: 1.6843 - val_accuracy: 0.7585
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0389 - accuracy: 0.9818 - val_loss: 1.9899 - val_accuracy: 0.7677


In [88]:
model_5_pred = tf.squeeze(tf.round(model_5.predict(x_test)))
model_5_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [89]:
calculate_results(y_test,model_5_pred)

{'Accuracy': 76.77165354330708,
 'F1_score': 0.7654097439388935,
 'Precision': 0.7696465472028295,
 'Recall': 0.7677165354330708}

# Model_6 : Transfer Learning from Tensorflow hub

In [105]:
from tensorflow.keras import layers
import tensorflow_hub as hub

embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                       input_shape=[], dtype=tf.string,
                       trainable=False)

model_6 = tf.keras.Sequential([
        embed,
        layers.Dense(64,activation='relu'),
        layers.Dense(1,activation='sigmoid')                       
])

In [102]:
model_6.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_3 (KerasLayer)   (None, 512)               256797824 
_________________________________________________________________
dense_20 (Dense)             (None, 64)                32832     
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [106]:
model_6.compile(optimizer=tf.keras.optimizers.Adam(),
                loss='binary_crossentropy',
                metrics = ['accuracy'])

In [107]:
history_6  = model_6.fit(x_train,y_train,epochs=10,
                         validation_data = (x_test,y_test),
                         callbacks=[create_tensorboard_callback(dir_name='training_logs',
                                                               experiment_name='model_6trans')]
                        )

Saving TensorBoard log files to: training_logs/model_6trans/20210521-170242
Epoch 1/10


Exception ignored in: <function CapturableResourceDeleter.__del__ at 0x7effb4a22680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/function.py", line 2969, in _get_concrete_function_internal_garbage_collected
    graph_function, _ = self._maybe_define_function(args, kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/p

215/215 [==============================] - 7s 19ms/step - loss: 0.5798 - accuracy: 0.7455 - val_loss: 0.4448 - val_accuracy: 0.7979
Epoch 2/10
215/215 [==============================] - 2s 10ms/step - loss: 0.4179 - accuracy: 0.8149 - val_loss: 0.4372 - val_accuracy: 0.8110
Epoch 3/10
215/215 [==============================] - 2s 10ms/step - loss: 0.4091 - accuracy: 0.8144 - val_loss: 0.4338 - val_accuracy: 0.8084
Epoch 4/10
215/215 [==============================] - 2s 10ms/step - loss: 0.3935 - accuracy: 0.8266 - val_loss: 0.4278 - val_accuracy: 0.8110
Epoch 5/10
215/215 [==============================] - 2s 10ms/step - loss: 0.3848 - accuracy: 0.8339 - val_loss: 0.4281 - val_accuracy: 0.8136
Epoch 6/10
215/215 [==============================] - 2s 10ms/step - loss: 0.3790 - accuracy: 0.8328 - val_loss: 0.4223 - val_accuracy: 0.8176
Epoch 7/10
215/215 [==============================] - 2s 10ms/step - loss: 0.3772 - accuracy: 0.8323 - val_loss: 0.4275 - val_accuracy: 0.8215
Epoch 8/10

In [108]:
model_6_pred = tf.squeeze(tf.round(model_6.predict(x_test)))
model_6_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [109]:
calculate_results(y_test,model_6_pred)

{'Accuracy': 81.62729658792651,
 'F1_score': 0.813022410660206,
 'Precision': 0.8252416515568405,
 'Recall': 0.8162729658792651}